모델을 공동으로 연구 개발하는데 공유하는 것
1. 모델을 만드는 코드
2. 모델의 훈련된 가중치 또는 파라미터

# 준비

## 라이브러리 호출

In [1]:
import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.4.1


## 데이터셋 로드

저장과 로드를 위한 작은 모델을 만들기 위해 1000개 의 데이터만 사용

In [2]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

## 모델링

In [3]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512, activation='relu', input_shape=(784, )),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])

    model.compile(
        optimizer='adam',
        loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'])

    return model

model = create_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


# 체크포인트 저장하기

tf.keras.callbacks.ModelCheckpoint 콜백을 사용하면 훈련 도중 또는 훈련 종료 시 모델을 지속적으로 저장할 수 있습니다.

In [4]:
checkpoint_path = "temp/training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# 새로운 콜백으로 모델 훈련하기
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # 콜백을 훈련에 전달합니다

# 옵티마이저의 상태를 저장하는 것과 관련되어 경고가 발생할 수 있습니다.
# 이 경고는 (그리고 이 노트북의 다른 비슷한 경고는) 이전 사용 방식을 권장하지 않기 위함이며 무시해도 좋습니다.

Epoch 1/10
32/32 [==============================] - 2s 12ms/step - loss: 1.6111 - accuracy: 0.5007 - val_loss: 0.7005 - val_accuracy: 0.7850

Epoch 00001: saving model to temp/training_1\cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 4ms/step - loss: 0.4678 - accuracy: 0.8869 - val_loss: 0.5264 - val_accuracy: 0.8320

Epoch 00002: saving model to temp/training_1\cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 4ms/step - loss: 0.2814 - accuracy: 0.9306 - val_loss: 0.4725 - val_accuracy: 0.8530

Epoch 00003: saving model to temp/training_1\cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 4ms/step - loss: 0.2263 - accuracy: 0.9550 - val_loss: 0.4495 - val_accuracy: 0.8560

Epoch 00004: saving model to temp/training_1\cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 4ms/step - loss: 0.1588 - accuracy: 0.9666 - val_loss: 0.4572 - val_accuracy: 0.8450

Epoch 00005: saving model to temp/training_1\cp.ckpt
Epoch 6/10
32/32 [===========

체크포인트의 디렉토리

In [17]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

# 체크포인트 불러오기

In [18]:
# 새로운 모델
model = create_model()
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("훈련되지 않은 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.3402 - accuracy: 0.1200
훈련되지 않은 모델의 정확도: 12.00%


In [19]:
# 가중치 로드
model.load_weights(checkpoint_path)

In [20]:
# 재평가
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4006 - accuracy: 0.8790
복원된 모델의 정확도: 87.90%


In [21]:
# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "temp/training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to temp/training_2\cp-0005.ckpt

Epoch 00010: saving model to temp/training_2\cp-0010.ckpt

Epoch 00015: saving model to temp/training_2\cp-0015.ckpt

Epoch 00020: saving model to temp/training_2\cp-0020.ckpt

Epoch 00025: saving model to temp/training_2\cp-0025.ckpt

Epoch 00030: saving model to temp/training_2\cp-0030.ckpt

Epoch 00035: saving model to temp/training_2\cp-0035.ckpt

Epoch 00040: saving model to temp/training_2\cp-0040.ckpt

Epoch 00045: saving model to temp/training_2\cp-0045.ckpt

Epoch 00050: saving model to temp/training_2\cp-0050.ckpt


In [23]:
# 가장 마지막 체크포인트 불러오기
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'temp/training_2\\cp-0050.ckpt'

In [24]:
# 새로운 모델 객체를 만듭니다
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4869 - accuracy: 0.8790
복원된 모델의 정확도: 87.90%


# 수동으로 가중치 저장하기

Model.save_weights 메서드를 사용하여 수동으로 가중치를 저장합니다.

기본적으로 tf.keras, 특히 save_weights는 .ckpt 확장자가 있는 TensorFlow 체크포인트 형식을 사용합니다

(.h5 확장자를 사용하여 HDF5에 저장하는 내용은 모델 저장 및 직렬화 가이드에서 다룸)

In [25]:
# 가중치를 저장합니다
model.save_weights('temp/checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('temp/checkpoints/my_checkpoint')

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4869 - accuracy: 0.8790
복원된 모델의 정확도: 87.90%


# 전체 모델 저장하기 / 불러오기

In [28]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
model.save('temp/saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 1s 5ms/step - loss: 1.5447 - accuracy: 0.5404
Epoch 2/5
32/32 [==============================] - 0s 3ms/step - loss: 0.4335 - accuracy: 0.8742
Epoch 3/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2934 - accuracy: 0.9250
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.2110 - accuracy: 0.9458
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1694 - accuracy: 0.9643
INFO:tensorflow:Assets written to: temp/saved_model/my_model\assets


In [31]:
new_model = tf.keras.models.load_model('temp/saved_model/my_model')
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))
print(new_model.predict(test_images).shape)

32/32 - 0s - loss: 0.4488 - accuracy: 0.8530
복원된 모델의 정확도: 85.30%
(1000, 10)


# HDF5 파일로 저장하기

케라스는 HDF5 표준을 따르는 기본 저장 포맷을 제공합니다.

In [34]:
# 새로운 모델 객체를 만들고 훈련합니다
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5 파일로 저장합니다
# '.h5' 확장자는 이 모델이 HDF5로 저장되었다는 것을 나타냅니다
model.save('temp/my_model.h5')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.5913 - accuracy: 0.5074
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4469 - accuracy: 0.8937
Epoch 3/5
32/32 [==============================] - 0s 3ms/step - loss: 0.3065 - accuracy: 0.9232
Epoch 4/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1925 - accuracy: 0.9531
Epoch 5/5
32/32 [==============================] - 0s 1ms/step - loss: 0.1425 - accuracy: 0.9730


In [35]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성합니다
new_model = tf.keras.models.load_model('temp/my_model.h5')
# 모델 구조를 출력합니다
new_model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_26 (Dense)             (None, 512)               401920    
_________________________________________________________________
dropout_13 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_27 (Dense)             (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [36]:
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print('복원된 모델의 정확도: {:5.2f}%'.format(100*acc))

32/32 - 0s - loss: 0.4284 - accuracy: 0.8550
복원된 모델의 정확도: 85.50%
